<a href="https://colab.research.google.com/github/Fahad-Blog/Data-Science-Portfolio/blob/main/Credit_Card_Approval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header = None)


In [ ]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [ ]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [ ]:
unique_val = {}
for col in cc_apps.columns:
    unique_values = cc_apps[col].unique()
    unique_val[col] = unique_values
unique_val


{0: array(['b', 'a', '?'], dtype=object),
 1: array(['30.83', '58.67', '24.50', '27.83', '20.17', '32.08', '33.17',
        '22.92', '54.42', '42.50', '22.08', '29.92', '38.25', '48.08',
        '45.83', '36.67', '28.25', '23.25', '21.83', '19.17', '25.00',
        '47.75', '27.42', '41.17', '15.83', '47.00', '56.58', '57.42',
        '42.08', '29.25', '42.00', '49.50', '36.75', '22.58', '27.25',
        '23.00', '27.75', '54.58', '34.17', '28.92', '29.67', '39.58',
        '56.42', '54.33', '41.00', '31.92', '41.50', '23.92', '25.75',
        '26.00', '37.42', '34.92', '34.25', '23.33', '23.17', '44.33',
        '35.17', '43.25', '56.75', '31.67', '23.42', '20.42', '26.67',
        '36.00', '25.50', '19.42', '32.33', '34.83', '38.58', '44.25',
        '44.83', '20.67', '34.08', '21.67', '21.50', '49.58', '27.67',
        '39.83', '?', '37.17', '25.67', '34.00', '49.00', '62.50', '31.42',
        '52.33', '28.75', '28.58', '22.50', '28.50', '37.50', '35.25',
        '18.67', '54.83', '

In [ ]:
categorical_col = [0,1,3,4,5,6,8,9,11]


In [ ]:
cc_apps[cc_apps[categorical_col]=='?'].count()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
11     0
12     0
13     0
dtype: int64

In [ ]:
categorical_col = [0,1,3,4,5,6,8,9,11]
for col in cc_apps[categorical_col]:
    mode_value = cc_apps[col].mode()[0]
    cc_apps[col].replace('?',mode_value,inplace=True)

In [ ]:
cc_apps[1] = pd.to_numeric(cc_apps[1],errors='coerce')
mean_value = cc_apps[1].mean()
cc_apps[1].fillna(mean_value,inplace=True)

In [ ]:
cc_apps[13] = cc_apps[13].replace('+',1)
cc_apps[13] = cc_apps[13].replace('-',0)

In [ ]:
categorical_col = [0,3,4,5,6,8,9,11]
X = cc_apps.drop(13,axis=1)
y = cc_apps[13]
X = pd.get_dummies(X,columns=categorical_col,drop_first=True)
X.head()
X.columns = X.columns.astype(str)

In [ ]:
#train test split
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=9)
X_train.head()

,1,2,7,10,12,0_b,3_u,3_y,4_gg,4_p,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s
107,25.00,11.000,4.500,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
538,34.25,1.750,0.250,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
423,26.17,0.835,1.165,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
50,25.75,0.500,0.875,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
479,26.50,2.710,0.085,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [ ]:
#scaling the variables
scaler  = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_scaled,y_train)
y_pred = lr.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[67 18]
 [ 4 49]]
              precision    recall  f1-score   support

           0       0.94      0.79      0.86        85
           1       0.73      0.92      0.82        53

    accuracy                           0.84       138
   macro avg       0.84      0.86      0.84       138
weighted avg       0.86      0.84      0.84       138



In [ ]:
# Hyper parameter tuning
param = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'], 'solver':['liblinear', 'lbfgs', 'sag', 'saga', 'newton-cg'],"C": np.linspace(0.001, 1.0, 10)}

logreg_cv = GridSearchCV(lr, param_grid = param, cv=5 )
logreg_cv.fit(X_train_scaled,y_train)

print("Best Parameter",logreg_cv.best_params_)
print("Best Parameter",logreg_cv.best_score_)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:

lr = LogisticRegression(C=0.445, penalty= 'l2', solver= 'saga')
lr.fit(X_train_scaled,y_train)
y_pred = lr.predict(X_test_scaled)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_pred, y_test))

[[67 18]
 [ 4 49]]
              precision    recall  f1-score   support

           0       0.94      0.79      0.86        85
           1       0.73      0.92      0.82        53

    accuracy                           0.84       138
   macro avg       0.84      0.86      0.84       138
weighted avg       0.86      0.84      0.84       138

0.8405797101449275


In [ ]:
# Hyper parameter tuning of comparitive model
from sklearn.tree import DecisionTreeClassifier

param = {'min_samples_leaf': [1, 5, 10, 25, 50, 100],
    'max_depth': [5, 10, 15, 20]}

logreg_cv = GridSearchCV(DecisionTreeClassifier(), param_grid = param, cv=5 )
logreg_cv.fit(X_train_scaled,y_train)

print("Best Parameter",logreg_cv.best_params_)
print("Best Parameter",logreg_cv.best_score_)

best_score = logreg_cv.best_score_

Best Parameter {'max_depth': 5, 'min_samples_leaf': 50}
Best Parameter 0.8642260442260443
